### Import Required Libraries and Set Up Environment Variables

In [265]:
# Dependencies
import requests
import time
from dotenv import load_dotenv, find_dotenv
import os
import pandas as pd
import json
import os
from datetime import datetime
## Load the NASA_API_KEY from the env file
load_dotenv(find_dotenv('keys.env'))
NASA_API_KEY = os.getenv('NASA_API_KEY')

### CME Data

In [266]:
# Set the base URL to NASA's DONKI API:
base_url = "https://api.nasa.gov/DONKI/"

# Set the specifier for CMEs:
CME = "CME"

# Search for CMEs published between a begin and end date
startDate = "2013-05-01"
endDate   = "2024-05-01"

# Build URL for CME
cme_url = f"{base_url}{CME}?startDate={startDate}&endDate={endDate}&api_key={NASA_API_KEY}"


In [267]:
# Make a "GET" request for the CME URL and store it in a variable named cme_response
cme_response = requests.get(cme_url)

# Check the response status
if cme_response.status_code == 200:
    print("Request was successful.")
else:
    print(f"Request failed with status code: {cme_response.status_code}")


Request was successful.


In [268]:
# Convert the response variable to json and store it as a variable named cme_json
cme_json = cme_response.json()


In [269]:
import json
# Preview the first result in JSON format
# Use json.dumps with argument indent=4 to format data
print(json.dumps(cme_json[0], indent=4))


{
    "activityID": "2013-05-01T03:12:00-CME-001",
    "catalog": "M2M_CATALOG",
    "startTime": "2013-05-01T03:12Z",
    "instruments": [
        {
            "displayName": "SOHO: LASCO/C2"
        },
        {
            "displayName": "SOHO: LASCO/C3"
        },
        {
            "displayName": "STEREO A: SECCHI/COR2"
        },
        {
            "displayName": "STEREO B: SECCHI/COR2"
        }
    ],
    "sourceLocation": "",
    "activeRegionNum": null,
    "note": "",
    "submissionTime": "2013-08-07T16:54Z",
    "versionId": 1,
    "link": "https://webtools.ccmc.gsfc.nasa.gov/DONKI/view/CME/2349/-1",
    "cmeAnalyses": [
        {
            "isMostAccurate": true,
            "time21_5": "2013-05-01T07:07Z",
            "latitude": 12.0,
            "longitude": -120.0,
            "halfAngle": 36.0,
            "speed": 860.0,
            "type": "C",
            "featureCode": "null",
            "imageType": null,
            "measurementTechnique": "null",
   

In [270]:
# Convert cme_json to a Pandas DataFrame 
cme_df = pd.DataFrame(cme_json)

# Keep only the columns: activityID, startTime, linkedEvents
cme_df = cme_df[['activityID', 'startTime', 'linkedEvents']]
cme_df.head()

,activityID,startTime,linkedEvents
0,2013-05-01T03:12:00-CME-001,2013-05-01T03:12Z,[{'activityID': '2013-05-04T04:52:00-IPS-001'}]
1,2013-05-02T05:24:00-CME-001,2013-05-02T05:24Z,None
2,2013-05-02T14:36:00-CME-001,2013-05-02T14:36Z,None
3,2013-05-03T18:00:00-CME-001,2013-05-03T18:00Z,None
4,2013-05-03T22:36:00-CME-001,2013-05-03T22:36Z,[{'activityID': '2013-05-07T04:37:00-IPS-001'}]


In [271]:
# Notice that the linkedEvents column allows us to identify the corresponding GST
# Remove rows with missing 'linkedEvents' since we won't be able to assign these to GSTs
cme_df = cme_df.dropna(subset=['linkedEvents'])
cme_df.head()

,activityID,startTime,linkedEvents
0,2013-05-01T03:12:00-CME-001,2013-05-01T03:12Z,[{'activityID': '2013-05-04T04:52:00-IPS-001'}]
4,2013-05-03T22:36:00-CME-001,2013-05-03T22:36Z,[{'activityID': '2013-05-07T04:37:00-IPS-001'}]
7,2013-05-09T19:29:00-CME-001,2013-05-09T19:29Z,[{'activityID': '2013-05-12T23:30:00-IPS-001'}]
10,2013-05-13T02:54:00-CME-001,2013-05-13T02:54Z,[{'activityID': '2013-05-13T01:53:00-FLR-001'}...
13,2013-05-13T16:18:00-CME-001,2013-05-13T16:18Z,[{'activityID': '2013-05-13T15:40:00-FLR-001'}...


In [272]:
# Notice that the linkedEvents sometimes contains multiple events per row
# Write a nested for loop that iterates first over each row in the cme DataFrame (using the index)
# and then iterates over the values in 'linkedEvents' 
# and adds the elements individually to a list of dictionaries where each row is one element 

# Initialize an empty list to store the expanded rows
expanded_rows = []

# Iterate over each index in the DataFrame
for i in cme_df.index:
    # Iterate over each dictionary in the list
    for event in cme_df.loc[i,'linkedEvents']:
        # Append a new dictionary to the expanded_rows list for each dictionary item and corresponding 'activityID' and 'startTime' value
        expanded_rows.append({
            'activityID': cme_df.at[i, 'activityID'],
            'startTime': cme_df.at[i, 'startTime'],
            'linkedEvent': event
        })

# Create a new DataFrame from the expanded rows
expanded_cme_df = pd.DataFrame(expanded_rows)
expanded_cme_df.head()

,activityID,startTime,linkedEvent
0,2013-05-01T03:12:00-CME-001,2013-05-01T03:12Z,{'activityID': '2013-05-04T04:52:00-IPS-001'}
1,2013-05-03T22:36:00-CME-001,2013-05-03T22:36Z,{'activityID': '2013-05-07T04:37:00-IPS-001'}
2,2013-05-09T19:29:00-CME-001,2013-05-09T19:29Z,{'activityID': '2013-05-12T23:30:00-IPS-001'}
3,2013-05-13T02:54:00-CME-001,2013-05-13T02:54Z,{'activityID': '2013-05-13T01:53:00-FLR-001'}
4,2013-05-13T02:54:00-CME-001,2013-05-13T02:54Z,{'activityID': '2013-05-13T04:12:00-SEP-001'}


In [273]:
# Create a function called extract_activityID_from_dict that takes a dict as input such as in linkedEvents
def extract_activityID_from_dict(event_dict):
        try:
                return event_dict.get('activityID', None)
        except AttributeError as e:
                print(f"Error extracting activityID: {e}")
                return None

# Verify the function works as expected using one row from linkedEvents as an example
example_event = cme_df['linkedEvents'].iloc[0][0]
print(extract_activityID_from_dict(example_event))

     # Log the error or print it for debugging



2013-05-04T04:52:00-IPS-001


In [274]:
# Apply this function to each row in the 'linkedEvents' column (you can use apply() and a lambda function)
# and create a new column called 'GST_ActivityID' using loc indexer:
cme_df['GST_ActivityID'] = cme_df['linkedEvents'].apply(lambda x: extract_activityID_from_dict(x[0]) if x else None)
cme_df.head()

,activityID,startTime,linkedEvents,GST_ActivityID
0,2013-05-01T03:12:00-CME-001,2013-05-01T03:12Z,[{'activityID': '2013-05-04T04:52:00-IPS-001'}],2013-05-04T04:52:00-IPS-001
4,2013-05-03T22:36:00-CME-001,2013-05-03T22:36Z,[{'activityID': '2013-05-07T04:37:00-IPS-001'}],2013-05-07T04:37:00-IPS-001
7,2013-05-09T19:29:00-CME-001,2013-05-09T19:29Z,[{'activityID': '2013-05-12T23:30:00-IPS-001'}],2013-05-12T23:30:00-IPS-001
10,2013-05-13T02:54:00-CME-001,2013-05-13T02:54Z,[{'activityID': '2013-05-13T01:53:00-FLR-001'}...,2013-05-13T01:53:00-FLR-001
13,2013-05-13T16:18:00-CME-001,2013-05-13T16:18Z,[{'activityID': '2013-05-13T15:40:00-FLR-001'}...,2013-05-13T15:40:00-FLR-001


In [275]:
# Remove rows with missing GST_ActivityID, since we can't assign them to GSTs:
cme_df = cme_df[cme_df['GST_ActivityID'].notna()]



In [276]:
# print out the datatype of each column in this DataFrame:
cme_df.info()


<class 'pandas.core.frame.DataFrame'>
Index: 1023 entries, 0 to 5523
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   activityID      1023 non-null   object
 1   startTime       1023 non-null   object
 2   linkedEvents    1023 non-null   object
 3   GST_ActivityID  1023 non-null   object
dtypes: object(4)
memory usage: 72.2+ KB


In [277]:
# Convert the 'GST_ActivityID' column to string format 
cme_df['GST_ActivityID'] = cme_df['GST_ActivityID'].astype(str)

# Convert startTime to datetime format  
cme_df['startTime'] = pd.to_datetime(cme_df['startTime'])

# Rename startTime to startTime_CME and activityID to cmeID
cme_df.rename(columns={'startTime': 'startTime_CME', 'activityID': 'cmeID'}, inplace=True)

# Drop linkedEvents
cme_df.drop(columns=['linkedEvents'], inplace=True)

# Verify that all steps were executed correctly
cme_df.info()


<class 'pandas.core.frame.DataFrame'>
Index: 1023 entries, 0 to 5523
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype              
---  ------          --------------  -----              
 0   cmeID           1023 non-null   object             
 1   startTime_CME   1023 non-null   datetime64[ns, UTC]
 2   GST_ActivityID  1023 non-null   object             
dtypes: datetime64[ns, UTC](1), object(2)
memory usage: 64.3+ KB


In [278]:
# We are only interested in CMEs related to GSTs so keep only rows where the GST_ActivityID column contains 'GST'
# use the method 'contains()' from the str library.  
cme_df = cme_df[cme_df['GST_ActivityID'].str.contains('GST', na=False)]
cme_df.head()

,cmeID,startTime_CME,GST_ActivityID
26,2013-06-02T20:24:00-CME-001,2013-06-02 20:24:00+00:00,2013-06-07T03:00:00-GST-001
1358,2015-09-04T14:12:00-CME-001,2015-09-04 14:12:00+00:00,2015-09-09T03:00:00-GST-001
1359,2015-09-04T19:24:00-CME-001,2015-09-04 19:24:00+00:00,2015-09-09T03:00:00-GST-001
1361,2015-09-06T23:30:00-CME-001,2015-09-06 23:30:00+00:00,2015-09-11T06:00:00-GST-001
1362,2015-09-07T10:36:00-CME-001,2015-09-07 10:36:00+00:00,2015-09-11T06:00:00-GST-001


### GST Data

In [279]:
# Set the base URL to NASA's DONKI API:
base_url = "https://api.nasa.gov/DONKI/"

# Set the specifier for Geomagnetic Storms (GST):
GST = "GST"

# Search for GSTs between a begin and end date
startDate = "2013-05-01"
endDate   = "2024-05-01"

# Build URL for GST
gst_url = f"{base_url}{GST}?startDate={startDate}&endDate={endDate}&api_key={NASA_API_KEY}"


In [280]:
# Make a "GET" request for the GST URL and store it in a variable named gst_response
gst_response = requests.get(gst_url)

# Check the response status
if gst_response.status_code == 200:
    print("Request was successful.")
else:
    print(f"Request failed with status code: {gst_response.status_code}")


Request was successful.


In [281]:
# Convert the response variable to json and store it as a variable named gst_json
gst_json = gst_response.json()

# Preview the first result in JSON format
# Use json.dumps with argument indent=4 to format data
print(json.dumps(gst_json[0], indent=4))


{
    "gstID": "2013-06-01T01:00:00-GST-001",
    "startTime": "2013-06-01T01:00Z",
    "allKpIndex": [
        {
            "observedTime": "2013-06-01T01:00Z",
            "kpIndex": 6.0,
            "source": "NOAA"
        }
    ],
    "link": "https://webtools.ccmc.gsfc.nasa.gov/DONKI/view/GST/326/-1",
    "linkedEvents": [
        {
            "activityID": "2013-05-31T15:45:00-HSS-001"
        }
    ],
    "submissionTime": "2013-07-15T19:26Z",
    "versionId": 1
}


In [282]:
# Convert gst_json to a Pandas DataFrame  
gst_df = pd.DataFrame(gst_json)

# Keep only the columns: activityID, startTime, linkedEvents
gst_df = gst_df[['gstID', 'startTime', 'linkedEvents']]
gst_df.head()

,gstID,startTime,linkedEvents
0,2013-06-01T01:00:00-GST-001,2013-06-01T01:00Z,[{'activityID': '2013-05-31T15:45:00-HSS-001'}]
1,2013-06-07T03:00:00-GST-001,2013-06-07T03:00Z,[{'activityID': '2013-06-02T20:24:00-CME-001'}]
2,2013-06-29T03:00:00-GST-001,2013-06-29T03:00Z,None
3,2013-10-02T03:00:00-GST-001,2013-10-02T03:00Z,[{'activityID': '2013-09-29T22:40:00-CME-001'}...
4,2013-12-08T00:00:00-GST-001,2013-12-08T00:00Z,[{'activityID': '2013-12-04T23:12:00-CME-001'}...


In [283]:
# Notice that the linkedEvents column allows us to identify the corresponding CME
# Remove rows with missing 'linkedEvents' since we won't be able to assign these to CME
gst_df = gst_df.dropna(subset=['linkedEvents'])
gst_df.head()

,gstID,startTime,linkedEvents
0,2013-06-01T01:00:00-GST-001,2013-06-01T01:00Z,[{'activityID': '2013-05-31T15:45:00-HSS-001'}]
1,2013-06-07T03:00:00-GST-001,2013-06-07T03:00Z,[{'activityID': '2013-06-02T20:24:00-CME-001'}]
3,2013-10-02T03:00:00-GST-001,2013-10-02T03:00Z,[{'activityID': '2013-09-29T22:40:00-CME-001'}...
4,2013-12-08T00:00:00-GST-001,2013-12-08T00:00Z,[{'activityID': '2013-12-04T23:12:00-CME-001'}...
5,2014-02-19T03:00:00-GST-001,2014-02-19T03:00Z,[{'activityID': '2014-02-16T14:15:00-CME-001'}...


In [284]:
# # Notice that the linkedEvents sometimes contains multiple events per row
# Use the explode method to ensure that each row is one element. Ensure to reset the index and drop missing values.
gst_df = gst_df.explode('linkedEvents').reset_index(drop=True).dropna(subset=['linkedEvents'])
gst_df.head()


,gstID,startTime,linkedEvents
0,2013-06-01T01:00:00-GST-001,2013-06-01T01:00Z,{'activityID': '2013-05-31T15:45:00-HSS-001'}
1,2013-06-07T03:00:00-GST-001,2013-06-07T03:00Z,{'activityID': '2013-06-02T20:24:00-CME-001'}
2,2013-10-02T03:00:00-GST-001,2013-10-02T03:00Z,{'activityID': '2013-09-29T22:40:00-CME-001'}
3,2013-10-02T03:00:00-GST-001,2013-10-02T03:00Z,{'activityID': '2013-10-02T01:54:00-IPS-001'}
4,2013-10-02T03:00:00-GST-001,2013-10-02T03:00Z,{'activityID': '2013-10-02T02:47:00-MPC-001'}


In [285]:
# Apply the extract_activityID_from_dict function to each row in the 'linkedEvents' column (you can use apply() and a lambda function)
# and create a new column called 'CME_ActivityID' using loc indexer:
gst_df['CME_ActivityID'] = gst_df['linkedEvents'].apply(lambda x: extract_activityID_from_dict(x) if x else None)

# Remove rows with missing CME_ActivityID, since we can't assign them to CMEs:
gst_df = gst_df[gst_df['CME_ActivityID'].notna()]
gst_df.head()

,gstID,startTime,linkedEvents,CME_ActivityID
0,2013-06-01T01:00:00-GST-001,2013-06-01T01:00Z,{'activityID': '2013-05-31T15:45:00-HSS-001'},2013-05-31T15:45:00-HSS-001
1,2013-06-07T03:00:00-GST-001,2013-06-07T03:00Z,{'activityID': '2013-06-02T20:24:00-CME-001'},2013-06-02T20:24:00-CME-001
2,2013-10-02T03:00:00-GST-001,2013-10-02T03:00Z,{'activityID': '2013-09-29T22:40:00-CME-001'},2013-09-29T22:40:00-CME-001
3,2013-10-02T03:00:00-GST-001,2013-10-02T03:00Z,{'activityID': '2013-10-02T01:54:00-IPS-001'},2013-10-02T01:54:00-IPS-001
4,2013-10-02T03:00:00-GST-001,2013-10-02T03:00Z,{'activityID': '2013-10-02T02:47:00-MPC-001'},2013-10-02T02:47:00-MPC-001


In [286]:
# Convert the 'CME_ActivityID' column to string format 
gst_df['CME_ActivityID'] = gst_df['CME_ActivityID'].astype(str)

# Check if 'activityID' column exists before converting
if 'activityID' in gst_df.columns:
    # Convert the 'activityID' column to string format 
    gst_df['activityID'] = gst_df['activityID'].astype(str)
else:
    print("Column 'activityID' does not exist in the DataFrame")

# Convert startTime to datetime format  
gst_df['startTime'] = pd.to_datetime(gst_df['startTime'])

# Rename startTime to startTime_GST 
gst_df.rename(columns={'startTime': 'startTime_GST', 'activityID': 'gstID'}, inplace=True)

# Drop linkedEvents
gst_df.drop(columns=['linkedEvents'], inplace=True)

# Verify that all steps were executed correctly
gst_df.info()



Column 'activityID' does not exist in the DataFrame
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 205 entries, 0 to 204
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype              
---  ------          --------------  -----              
 0   gstID           205 non-null    object             
 1   startTime_GST   205 non-null    datetime64[ns, UTC]
 2   CME_ActivityID  205 non-null    object             
dtypes: datetime64[ns, UTC](1), object(2)
memory usage: 4.9+ KB


In [287]:
# We are only interested in GSTs related to CMEs so keep only rows where the CME_ActivityID column contains 'CME'
# use the method 'contains()' from the str library.  
gst_df = gst_df[gst_df['CME_ActivityID'].str.contains('CME', na=False)]
gst_df.head()

,gstID,startTime_GST,CME_ActivityID
1,2013-06-07T03:00:00-GST-001,2013-06-07 03:00:00+00:00,2013-06-02T20:24:00-CME-001
2,2013-10-02T03:00:00-GST-001,2013-10-02 03:00:00+00:00,2013-09-29T22:40:00-CME-001
5,2013-12-08T00:00:00-GST-001,2013-12-08 00:00:00+00:00,2013-12-04T23:12:00-CME-001
7,2014-02-19T03:00:00-GST-001,2014-02-19 03:00:00+00:00,2014-02-16T14:15:00-CME-001
9,2014-02-20T03:00:00-GST-001,2014-02-20 03:00:00+00:00,2014-02-18T01:25:00-CME-001


### Merge both datatsets

In [288]:
# Merge both datasets using 'gstID' and 'CME_ActivityID' for gst and 'GST_ActivityID' and 'cmeID' for cme. Use the 'left_on' and 'right_on' specifiers.
merged_df = pd.merge(gst_df, cme_df, left_on=['gstID', 'CME_ActivityID'], right_on=['GST_ActivityID', 'cmeID'])
merged_df


,gstID,startTime_GST,CME_ActivityID,cmeID,startTime_CME,GST_ActivityID
0,2013-06-07T03:00:00-GST-001,2013-06-07 03:00:00+00:00,2013-06-02T20:24:00-CME-001,2013-06-02T20:24:00-CME-001,2013-06-02 20:24:00+00:00,2013-06-07T03:00:00-GST-001
1,2015-09-09T03:00:00-GST-001,2015-09-09 03:00:00+00:00,2015-09-04T14:12:00-CME-001,2015-09-04T14:12:00-CME-001,2015-09-04 14:12:00+00:00,2015-09-09T03:00:00-GST-001
2,2015-09-09T03:00:00-GST-001,2015-09-09 03:00:00+00:00,2015-09-04T19:24:00-CME-001,2015-09-04T19:24:00-CME-001,2015-09-04 19:24:00+00:00,2015-09-09T03:00:00-GST-001
3,2015-09-11T06:00:00-GST-001,2015-09-11 06:00:00+00:00,2015-09-06T23:30:00-CME-001,2015-09-06T23:30:00-CME-001,2015-09-06 23:30:00+00:00,2015-09-11T06:00:00-GST-001
4,2015-09-11T06:00:00-GST-001,2015-09-11 06:00:00+00:00,2015-09-07T10:36:00-CME-001,2015-09-07T10:36:00-CME-001,2015-09-07 10:36:00+00:00,2015-09-11T06:00:00-GST-001
5,2024-03-24T12:00:00-GST-001,2024-03-24 12:00:00+00:00,2024-03-23T01:48:00-CME-001,2024-03-23T01:48:00-CME-001,2024-03-23 01:48:00+00:00,2024-03-24T12:00:00-GST-001


In [289]:
# Verify that the new DataFrame has the same number of rows as cme and gst
print(f"Number of rows in CME DataFrame: {len(cme_df)}")
print(f"Number of rows in GST DataFrame: {len(gst_df)}")
print(f"Number of rows in merged DataFrame: {len(merged_df)}")
merged_df.info()

Number of rows in CME DataFrame: 6
Number of rows in GST DataFrame: 61
Number of rows in merged DataFrame: 6
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype              
---  ------          --------------  -----              
 0   gstID           6 non-null      object             
 1   startTime_GST   6 non-null      datetime64[ns, UTC]
 2   CME_ActivityID  6 non-null      object             
 3   cmeID           6 non-null      object             
 4   startTime_CME   6 non-null      datetime64[ns, UTC]
 5   GST_ActivityID  6 non-null      object             
dtypes: datetime64[ns, UTC](2), object(4)
memory usage: 416.0+ bytes


### Computing the time it takes for a CME to cause a GST

In [290]:
# Compute the time diff between startTime_GST and startTime_CME by creating a new column called `timeDiff`.
merged_df['timeDiff'] = merged_df['startTime_GST'] - merged_df['startTime_CME']
merged_df.head()

,gstID,startTime_GST,CME_ActivityID,cmeID,startTime_CME,GST_ActivityID,timeDiff
0,2013-06-07T03:00:00-GST-001,2013-06-07 03:00:00+00:00,2013-06-02T20:24:00-CME-001,2013-06-02T20:24:00-CME-001,2013-06-02 20:24:00+00:00,2013-06-07T03:00:00-GST-001,4 days 06:36:00
1,2015-09-09T03:00:00-GST-001,2015-09-09 03:00:00+00:00,2015-09-04T14:12:00-CME-001,2015-09-04T14:12:00-CME-001,2015-09-04 14:12:00+00:00,2015-09-09T03:00:00-GST-001,4 days 12:48:00
2,2015-09-09T03:00:00-GST-001,2015-09-09 03:00:00+00:00,2015-09-04T19:24:00-CME-001,2015-09-04T19:24:00-CME-001,2015-09-04 19:24:00+00:00,2015-09-09T03:00:00-GST-001,4 days 07:36:00
3,2015-09-11T06:00:00-GST-001,2015-09-11 06:00:00+00:00,2015-09-06T23:30:00-CME-001,2015-09-06T23:30:00-CME-001,2015-09-06 23:30:00+00:00,2015-09-11T06:00:00-GST-001,4 days 06:30:00
4,2015-09-11T06:00:00-GST-001,2015-09-11 06:00:00+00:00,2015-09-07T10:36:00-CME-001,2015-09-07T10:36:00-CME-001,2015-09-07 10:36:00+00:00,2015-09-11T06:00:00-GST-001,3 days 19:24:00


In [291]:
# Use describe() to compute the mean and median time 
# that it takes for a CME to cause a GST.
time_diff_stats = merged_df['timeDiff'].describe()
time_diff_stats


count                            6
mean               3 days 18:31:00
std      1 days 04:10:07.703849362
min                1 days 10:12:00
25%                3 days 22:10:30
50%                4 days 06:33:00
75%                4 days 07:21:00
max                4 days 12:48:00
Name: timeDiff, dtype: object

### Exporting data in csv format

In [292]:
# Export data to CSV without the index
merged_df.to_csv('merged_data.csv', index=False)
